In [ ]:
from pyairtable import Table
from pyairtable import Api
import pandas as pd
from tqdm import tqdm
from pyairtable.formulas import match
import numpy as np
import requests
import time
import pprint
from gspread_dataframe import set_with_dataframe
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import os

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env

AIRTABLE_API_KEY = os.getenv('AIRTABLE_API_KEY')
BASE_ID = os.getenv('BASE_ID')
REGRID_API_KEY = os.getenv('REGRID_API_KEY')

In [ ]:
# TABLE_NAME = 'Properties'
# headers = {
#     "Authorization": f"Bearer {AIRTABLE_API_KEY}",
#     "Content-Type": "application/json"
# }

# response = requests.get("https://api.airtable.com/v0/meta/bases", headers=headers)

# if response.status_code == 200:
#     bases = response.json().get("bases", [])
#     for b in bases:
#         print(f"Name: {b['name']} | ID: {b['id']}")
# else:
#     print(f"Error {response.status_code}: {response.text}")

## 🔍 Step 1: Pull APNs from Airtable Properties Table  
We retrieve a list of APNs from the "Properties" table using the Airtable API. This will serve as our input for fetching parcel data from Regrid.

In [ ]:
# Create API and access the table
TABLE_NAME = 'Regrid'
api = Api(AIRTABLE_API_KEY)
table = api.table(BASE_ID, TABLE_NAME)

# Get records and extract APNs
records = table.all()
apns = [record['fields'].get('APN') for record in records if 'APN' in record['fields']]

print(f"Pulled {len(apns)} APNs from Airtable.")

## 🌐 Step 2: Fetch Parcel Data from Regrid API  
This function queries the Regrid API for each APN collected and returns raw parcel data.

In [ ]:
# def get_parcel_by_apn(apn):
#     url = f"https://app.regrid.com/api/v2/parcels/apn?parcelnumb={apn}&token={REGRID_API_KEY}"
#     headers = {"accept": "application/json"}

#     response = requests.get(url, headers=headers)

#     if response.status_code == 200:
#         data = response.json()

#         # Check for parcels > features > list of parcels
#         features = data.get('parcels', {}).get('features', [])
#         if features:
#             print(response.text)
#             return features[0]  # This is the full parcel feature (geometry + properties)
#         else:
#             print(f"APN {apn}: No parcel found.")
#             return None
#     else:
#         print(f"APN {apn}: Error {response.status_code} - {response.text}")
#         return None

In [ ]:
def get_parcel_by_apn(apn):
    url = f"https://app.regrid.com/api/v2/parcels/apn?parcelnumb={apn}&token={REGRID_API_KEY}"
    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()

        # Check for parcels > features > list of parcels
        features = data.get('parcels', {}).get('features', [])
        if features:
            print(f"✅ Found parcel for APN {apn}")
            return features[0]  # This is the full parcel feature (geometry + properties)
        else:
            print(f"⚠️ No parcel found for APN {apn}")
            return None
    else:
        print(f"❌ Error for APN {apn}: Status {response.status_code} - {response.text}")
        return None

### 🔄 Step 3: Extract Required Fields from Regrid API Response  
We define `extract_parcel_fields()` to format and filter the data returned by Regrid to only include the fields we care about.

In [ ]:
def extract_parcel_fields(parcel):
    fields = parcel['properties']['fields']

    return {
        'APN': fields.get('parcelnumb'),
        'Latitude': fields.get('lat'),
        'Longitude': fields.get('lon'),
        'Owner': fields.get('owner'),
        'Use Code': fields.get('usecode'),
        'Use Description': fields.get('usedesc'),
        'Zoning': fields.get('zoning'),
        'Zoning Description': fields.get('zoning_description'),
        'Last Sale Date': fields.get('saledate'),
        'Last Sale Price': fields.get('saleprice'),
        'Year Built': fields.get('yearbuilt'),
        'Property Address': fields.get('address'),
        'City (Physical)': fields.get('scity'),
        # 'City (Mailing)': fields.get('mail_city'),
        'City (County Reported)': fields.get('city'),
        'State': fields.get('state2'),
        'County': fields.get('county'),
        'ZIP Code': fields.get('szip5'),
        'Lot SqFt': fields.get('ll_gissqft'),
        'Lot Acres': fields.get('ll_gisacre'),
        'Subdivision': fields.get('subdivision'),
        'Qualified Opportunity Zone': fields.get('qoz'),
    }

In [ ]:
# ## TESTING READOUT

# test_apn = "50183176"  # Replace with any APN you'd like to test
# parcel = get_parcel_by_apn(test_apn)

# if parcel:
#     data = extract_parcel_fields(parcel)
#     pprint.pprint(data)

## 🧹 Step 4: Process Each APN and Store Clean Data  
For each APN, we get raw data from Regrid using `get_parcel_by_apn`, clean it using `extract_parcel_fields`, and append the result to `all_parcel_data`.

In [ ]:
all_parcel_data = []

for apn in apns:
    parcel = get_parcel_by_apn(apn)
    if parcel:
        data = extract_parcel_fields(parcel)
        all_parcel_data.append(data)
    
    time.sleep(0.25)  # small delay to avoid rate limits

## ✅ Step 5: Validate and Preview Parcel Data  
Quick visual inspection of the processed parcel data before uploading to Airtable.

In [ ]:
all_parcel_data

In [ ]:
pprint.pprint(all_parcel_data[0])

## 📄 Step 6: Convert List to DataFrame

Convert the list of dictionaries (all_parcel_data) into a Pandas DataFrame to allow for easier cleaning, transformation, and export.

In [ ]:
df = pd.DataFrame(all_parcel_data)
df.head()

## 🏷️ Step 7: Add Currently Owned Column

Create a new column that flags whether each property is currently owned.
We do this by checking if the Owner field contains any known ownership keywords

In [ ]:
ownership_keywords = [kw.upper() for kw in os.getenv("OWNERSHIP_KEYWORDS").split(",")]


# Step 3: Function to check if any keyword is in the owner string
def is_currently_owned(owner):
    if not isinstance(owner, str):
        return "No"
    owner_upper = owner.upper()
    return "Yes" if any(keyword in owner_upper for keyword in ownership_keywords) else "No"

# Step 4: Apply the function to create the new column
df["Currently Owned"] = df["Owner"].apply(is_currently_owned)

# Optional: Preview results
# Show all rows but only the APN, Owner, and Currently Owned columns
print(df[["APN", "Owner", "Currently Owned"]].to_string(index=False))

In [ ]:
df.head()

In [ ]:
print(df.dtypes)

## 💾 Step 8: Save DataFrame to CSV

Save the enriched and filtered parcel data to a local CSV file.
This lets us re-load it later without consuming additional API credits.

In [ ]:
df.to_csv("regrid_output.csv", index=False)
print("Saved parcel data to regrid_output.csv")

## 📂 Step 9: Reload Data from CSV

Load the CSV file back into a DataFrame and apply strict column data type mappings (dtype_mapping) to ensure compatibility with Airtable.

### Mapping data types with `dtype_mapping` so they read in correctly to match airtable upload

In [ ]:
dtype_mapping = {
    'APN': str,
    'Latitude': float,
    'Longitude': float,
    'Owner': str,
    'Use Code': str,
    'Use Description': str,
    'Zoning': str,
    'Zoning Description': str,
    'Last Sale Date': str,
    'Last Sale Price': str,
    'Year Built': str,  # Changed to nullable integer
    'Property Address': str,
    'City (Physical)': str,
    'City (County Reported)': str,
    'State': str,
    'County': str,
    'ZIP Code': str,  # Keep ZIPs as strings
    'Lot SqFt': 'Int64',  # Already set correctly
    'Lot Acres': float,
    'Subdivision': str,
    'Qualified Opportunity Zone': str,
    'Currently Owned': str
}

In [ ]:
df = pd.read_csv('regrid_output.csv', dtype=dtype_mapping)
df.head()

In [ ]:
df[df["Currently Owned"] == "No"].count()

In [ ]:
df = df[df["Currently Owned"] == "Yes"]

print(df[["APN", "Owner", "Currently Owned"]].to_string(index=False))

In [ ]:
# Remove all hyphens from the APN column
df["APN"] = df["APN"].astype(str).str.replace("-", "", regex=False)
df.head()

In [ ]:
print(df.dtypes)

In [ ]:
api = Api(AIRTABLE_API_KEY)
regrid_table = api.table(BASE_ID, "Regrid Output")

In [ ]:
# test upload
# try:
#     regrid_table.create(all_parcel_data[0])
#     print("✅ Successfully uploaded first record to Airtable.")
# except Exception as e:
#     print(f"❌ Error uploading record: {e}")

In [ ]:
# print("\n".join([f'- [ ] {field}' for field in all_parcel_data[0].keys()]))

## 🧹 Step 10: Clean DataFrame (Replace NaN with None)

Convert all NaN values in the DataFrame to None to ensure compatibility with JSON serialization when uploading to Airtable.

In [ ]:
df.head()

In [ ]:
# Convert NaN to None (to avoid JSON errors)
df_cleaned = df.where(pd.notnull(df), None)
# df_cleaned = df.replace({np.nan: None})
# df_cleaned = df.map(lambda x: None if pd.isna(x) else x)
# Replace NaN and <NA> in "Last Sale Price" and "Year Built" with None
# for col in ["Last Sale Price", "Year Built"]:
#     df_cleaned[col] = df_cleaned[col].apply(lambda x: None if pd.isna(x) else x)
df_cleaned.head()

### Checking data types again before uploading

In [ ]:
print(df_cleaned.dtypes)

### Converting the cleaned df, `df_cleaned` to a list of dictionaries to upload to airtable: `records_to_upload`

In [ ]:
records_to_upload = df_cleaned.to_dict(orient="records")
pprint.pprint(records_to_upload[0])

In [ ]:
# Show rows where Latitude or Longitude is NaN
# df[df['Latitude'].isna() | df['Longitude'].isna()]

## 🔁 Step 13: Update or Create Records in Airtable

Loop through all records and upload to Airtable:
	•	If a record with the same APN already exists, update it.
	•	If it doesn’t exist, create a new one.
This ensures deduplication and allows the table to stay synced.

In [ ]:
for record in records_to_upload:
    apn = record.get("APN")

    try:
        # Look for an existing record with matching APN
        existing = regrid_table.first(formula=match({"APN": apn}))

        if existing:
            # Update existing record
            regrid_table.update(existing['id'], record)
            print(f"🔁 Updated record for APN: {apn}")
        else:
            # Create new record
            regrid_table.create(record)
            print(f"✅ Created new record for APN: {apn}")

        time.sleep(0.25)  # Prevent hitting Airtable's rate limit

    except Exception as e:
        print(f"❌ Error processing APN {apn}: {e}")

## 🔁 Step 14: Pull Data from Airtable (Regrid Output Table)
We treat Airtable's **Regrid Output** table as our **source of truth**. This table contains only the currently owned properties and includes both automated Regrid fields and manually entered fields like `Insurance`, `Units`, `Property Type`, and more.

We'll pull the data from Airtable so we can push it to Google Sheets for visualization in Looker Studio.

In [431]:
TABLE = "Regrid"

table = Table(AIRTABLE_API_KEY, BASE_ID, TABLE)

# Fetch all records
records = table.all()

# Convert to DataFrame
airtable_data = pd.DataFrame([record['fields'] for record in records])
airtable_data.head()

/var/folders/6d/4lz7w_g12fn5bdx81b02gdyc0000gn/T/ipykernel_92645/2805409874.py:3: DeprecationWarning: Passing API keys or base IDs to pyairtable.Table is deprecated; use Api.table() or Base.table() instead. See https://pyairtable.rtfd.org/en/latest/migrations.html for details.
  table = Table(AIRTABLE_API_KEY, BASE_ID, TABLE)


,APN,Property Address,City (Physical),State,County,ZIP Code,Owner,Latitude,Longitude,Use Code,Use Description,Zoning,Zoning Description,Last Sale Date,Last Sale Price,Year Built,City (County Reported),Lot SqFt,Lot Acres,Subdivision,Qualified Opportunity Zone,Currently Owned
0,11618113,1438 E POLK ST,PHOENIX,AZ,maricopa,85006,MI CASA PROPERTY LLC,33.453107,-112.050640,8613,ONE SFR / ONE TRIPLEX,R-4,Multiple Family Residence,2013-06-07,237960.0,1910.0,phoenix,23709,0.54427,"GERMANIA PLACE LOTS 4-11,15-19,21,22,31-36",No,Yes
1,505311480,1149 E 4TH ST,CASA GRANDE,AZ,pinal,85122,MI CASA PROPERTY LTD LBLTY CO,32.876370,-111.735920,0121,Residential,R-1,Single Family Residential,2010-10-04,136250.0,1987.0,casa-grande,5224,0.11992,NaN,No,Yes
2,5328022066,1330 OAK VIEW AVE,SAN MARINO,CA,los-angeles,91108,"HSU,MARY M",34.124177,-118.118688,0101,Residential - Single,R-1-I,Single Family Dwelling District I,1998-06-24,1010010.0,2000.0,pasadena,25534,0.58616,NaN,No,Yes
3,4453036004,NaN,NaN,CA,los-angeles,90265,MI CASA HOLDINGS LTD,34.051800,-118.651669,010V,Residential - Single,R-C-20,Rural Coastal,2010-12-09,110000.0,NaN,agoura-hills-malibu,433524,9.95214,NaN,No,Yes
4,0253204080000,18889 VALLEY BLVD,BLOOMINGTON,CA,san-bernardino,92316,MI CASA PROPERTY LLC,34.070157,-117.392596,510,SFR,VC/COM,Valley Corridor Specific Plan Commercial,2010-09-28,67000.0,1912.0,san-bernardino,6487,0.14892,NaN,Yes,Yes


## 📤 Step 15: Upload Data to Google Sheets
We use the `gspread` and `gspread_dataframe` libraries to upload the cleaned DataFrame to a specific worksheet in Google Sheets.

We **clear the sheet** first to avoid duplicates, then upload the latest Airtable data. This Google Sheet will serve as the data source for our Looker Studio dashboard.

In [ ]:
# Load credentials from JSON
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("google_creds.json", scope)
client = gspread.authorize(creds)

# Open the sheet
worksheet = client.open("Airtable Regrid Output").worksheet("Sheet1")

# Clear and upload data
worksheet.clear()
set_with_dataframe(worksheet, airtable_data)

print("✅ Airtable data synced to Google Sheets.")